In [1]:
import numpy as np
import matplotlib.pyplot as pl
import scipy.optimize as sop
import scipy.spatial as ssp
from ipywidgets import StaticInteract, RangeWidget, RadioWidget
%pylab inline


Populating the interactive namespace from numpy and matplotlib


# A brief introduction to Gaussian process regression 

## Suzanne Aigrain, University of Oxford

### Birmingham Astrostatistics Tutorials, 9 July 2015

This document is intended as a basic introduction to Gaussian Process (GP) regression, with ultra-simple examples. All the GP code is built from scratch, to illustrate how simple it can be. The intended audience is astronomy grad students.

## What are GPs used for?

GPs are used for classification and regression from finance to zoology. 

Useful for  data containing non-trivial stochastic signals or noise: we cannot write down and expression for $y = f(x)$, we expect $f$ to have certain properties such as smoothness, amplitude, scale, and/or periodicity. 

Example: time-series data. Causation implies correlation, so time-series data nearly always has correlated noise.

## What is a GP?

A Gaussian process is a collection of random variables, any 
finite number of which have a joint Gaussian distribution.

Consider a scalar variable $y$, drawn from a Gaussian distribution with mean $\mu$ and variance $\sigma^2$:

$$
p(y) = \frac{1}{\sqrt{2 \pi} \sigma} \exp \left[ - \frac{(y-\mu)^2}{2 \sigma^2} \right].
$$

As a short hand, we write: $y \sim \mathcal{N}(\mu,\sigma^2)$.

In [2]:
def gauss1d(x,mu,sig):
    return np.exp(-(x-mu)**2/sig*2/2.)/np.sqrt(2*np.pi)/sig
def pltgauss1d(sig=1):
    mu=0
    x = np.r_[-4:4:101j]
    fig = pl.figure()
    pl.plot(x, gauss1d(x,mu,sig),'k-');
    pl.axvline(mu,c='k',ls='-');
    pl.axvline(mu+sig,c='k',ls='--');
    pl.axvline(mu-sig,c='k',ls='--');
    pl.axvline(mu+2*sig,c='k',ls=':');
    pl.axvline(mu-2*sig,c='k',ls=':');
    pl.xlim(x.min(),x.max());
    pl.ylim(0,1);
    pl.xlabel(r'$y$');
    pl.ylabel(r'$p(y)$');
    return fig
StaticInteract(pltgauss1d,
               sig=RangeWidget(0.5, 2.0, 0.25,name=r'$\sigma$',default=1))

Now let us consider a pair of variables $y_1$ and $y_2$, drawn from a *bivariate Gaussian distribution*. The *joint probability density* for $y_1$ and $y_2$ is:

$$
\left[ \begin{array}{l} y_1 \\ y_2 \end{array} \right] \sim \mathcal{N} \left(
\left[ \begin{array}{l} \mu_1 \\ \mu_2 \end{array}  \right] , 
\left[ \begin{array}{ll} 
\sigma_1^2 & C \\
C & \sigma_2^2 
\end{array}  \right] 
\right),
$$

where $C = {\rm cov}(y_1,y_2)$ is the *covariance* between $y_1$ and $y_2$.
The second term on the right hand side is the *covariance matrix*, $K$.

We now use two powerful identities of Gaussian distributions to elucidate the relationship between $y_1$ and $y_2$. 

The marginal distribution of $y_1$ describes what we know about $y_1$ in the absence of any other information about $y_2$ and is simply:

$$
p(y_1)= \mathcal{N} (\mu_1,\sigma_1^2).
$$

If we know the value of $y_2$, the probability density for $y_1$ collapses to the the *conditional distribution* of $y_1$ given $y_2$:

$$
p(y_1 \mid y_2) = \mathcal{N} \left( \mu_1 + C (y_2-\mu_2)/\sigma_2^2, \sigma_1^2-C^2\sigma_2^2 \right).
$$

If $K$ is diagonal, i.e. if $C=0$, we can immediately see that $p(y_1 \mid y_2) = p(y_1)$, i.e. measuring $y_2$ doesn't teach us anything about $y_1$. The two variables are said to be *uncorrelated*. 

If the variables are *correlated*, $C \neq 0$, then measuring $y_2$ does alter our knowledge of $y_1$: it modifies the mean and reduces the variance.

In [3]:
 def gauss2d(x1,x2,mu1,mu2,sig1,sig2,rho):
    z = (x1-mu1)**2/sig1**2 + (x2-mu2)**2/sig2**2 - 2*rho*(x1-mu1)*(x2-mu2)/sig1/sig2
    e = np.exp(-z/2/(1-rho**2))
    return e/(2*np.pi*sig1*sig2*sqrt(1-rho**2))
def pltgauss2d(rho=0,show_cond=0):
    mu1, sig1 = 0,1
    mu2, sig2 = 0,1
    y2o = -1
    x1 = np.r_[-4:4:101j]
    x2 = np.r_[-4:4:101j]
    x22d,x12d = np.mgrid[-4:4:101j,-4:4:101j]
    y = gauss2d(x12d,x22d,mu1,mu2,sig1,sig2,rho)
    y1 = gauss1d(x1,mu1,sig1)
    y2 = gauss1d(x2,mu2,sig2)
    mu12 = mu1+rho*(y2o-mu2)/sig2**2
    sig12 = np.sqrt(sig1**2-rho**2*sig2**2)
    y12 = gauss1d(x1,mu12,sig12)
    fig=pl.figure(figsize=(6,6))
    ax1 = pl.subplot2grid((3,3),(1,0),colspan=2,rowspan=2,aspect='equal')
    v = np.array([0.02,0.1,0.3,0.6]) * y.max()
    CS = pl.contour(x1,x2,y,v,colors='k')
    if show_cond: pl.axhline(y2o,c='r')
    pl.xlabel(r'$y_1$');
    pl.ylabel(r'$y_2$');
    pl.xlim(x1.min(),x1.max())
    ax1.xaxis.set_major_locator(plt.MaxNLocator(5, prune = 'both'))
    ax1.yaxis.set_major_locator(plt.MaxNLocator(5, prune = 'both'))
    ax2 = pl.subplot2grid((3,3),(0,0),colspan=2,sharex=ax1)
    pl.plot(x1,y1,'k-')
    if show_cond: pl.plot(x1,y12,'r-')
    pl.ylim(0,0.8)
    plt.ylabel(r'$p(y_1)$')
    pl.setp(ax2.get_xticklabels(), visible=False)
    ax2.xaxis.set_major_locator(plt.MaxNLocator(5, prune = 'both'))
    ax2.yaxis.set_major_locator(plt.MaxNLocator(4, prune = 'upper'))
    pl.xlim(x1.min(),x1.max())
    ax3 = pl.subplot2grid((3,3),(1,2),rowspan=2,sharey=ax1)
    pl.plot(y2,x2,'k-')
    if show_cond: plt.axhline(y2o,c='r')
    pl.ylim(x2.min(),x2.max());
    pl.xlim(0,0.8);
    plt.xlabel(r'$p(y_2)$')
    pl.setp(ax3.get_yticklabels(), visible=False)
    ax3.xaxis.set_major_locator(plt.MaxNLocator(4, prune = 'upper'))
    ax3.yaxis.set_major_locator(plt.MaxNLocator(5, prune = 'both'))
    fig.subplots_adjust(hspace=0,wspace=0)
    return fig
StaticInteract(pltgauss2d, \
               rho=RangeWidget(-0.5,0.5,0.25,name=r'$\rho$',default=0), \
               show_cond=RadioWidget([0,1],default=0))

Now consider $N$ variables drawn from a multivariate Gaussian distribution:

$$
\boldsymbol{y} \sim \mathcal{N} (\boldsymbol{\mu},K)
$$

where $y = (y_1,y_2,\ldots,y_N)^T$, $\mu = (\mu_1,\mu_2,\ldots,\mu_N)^T$, and $K$ is an $N \times N$ positive semi-definite matrix with elements $K_{ij}={\rm cov}(y_i,y_j)$. 

A Gaussian process is an extension of this concept to infinite $N$, giving rise to a probability distribution over functions. 

This last generalisation may not be obvious, but in practice we get around it by only ever dealing with finite samples drawn from a GP.

## Specifying a GP

A Gaussian process is completely specified by its mean function and covariance function. We define the mean function $m(x)$ and the covariance function $k(x,x)$ of a real process $f(x)$ as

$$
\begin{array}{rcl}
m(x) & = & \mathrm{E}[f(x)], \\
k(x,x') & = & E[(f(x) − m(x))(f(x' − m(x'))],
\end{array}
$$

and will write the Gaussian process as

$$
f(x) \sim \mathcal{GP}(m(x), k(x,x'))
$$

(For simplicity we have assumed the input $x$ is one-dimensional -- for example, it might represent time. However, note that in general the input space can have more than one dimension. We will see an example of a GP with multi-dimensional inputs later.)

When modelling data using a GP, the mean function represents the deterministic component of the model, and the covariance function encodes the stochastic component. 

For example, if modelling the light curve of a star with a transiting planet, the mean function might be a planetary transit model and the covariance function might describe stellar variability and/or observational noise.

Once we have selected a mean and a covariance function, we can select a finite set of inputs $\boldsymbol{x}$ and evaluate the mean vector $\boldsymbol{m}$:

$$
\boldsymbol{m}=m(\boldsymbol{x})
$$ 

and the covariance matrix $K$:

$$
K_{ij} = k(x_i,x_j)
$$

at these inputs. 

The joint probability distribution over the outputs $\boldsymbol{y}$ is then

$$
p(\boldsymbol{y} \mid m,k) = \mathcal{N}( \boldsymbol{m},K).
$$

Note that strictly speaking the left hand side should read $p(\boldsymbol{y} \mid \boldsymbol{x},m,k)$, but we have omitted the inputs for the sake of conciseness, and will do so for the reminder of this document.

(For simplicity, we will assume the mean function is zero everywhere. We will return to non-zero mean functions later.)

## The squared exponential covariance function

The simplest, most widely used covariance function is the squared exponential:

$$
k_{\rm SE}(x,x')=A^2 \exp \left[ - \frac{r^2}{2 l^2} \right]
$$

where $r=|x-x'|$.

This gives rise to smooth functions with input scale (characteristic length scale) $A$ and output scale (amplitude) $l$.

In [4]:
# Kernel definitions

def kernel_SE(X1,X2,par):
    p0 = 10.0**par[0]
    p1 = 10.0**par[1]
    D2 = ssp.distance.cdist(X1,X2,'sqeuclidean')
    K = p0**2 * np.exp(- D2 / 2. / p1**2)
    return np.matrix(K)
def kernel_Mat32(X1,X2,par):
    p0 = 10.0**par[0]
    p1 = 10.0**par[1]
    DD = ssp.distance.cdist(X1, X2, 'euclidean')
    arg = np.sqrt(3) * abs(DD) / p1
    K = p0**2 * (1 + arg) * np.exp(- arg)
    return np.matrix(K)
def kernel_RQ(X1,X2,par):
    p0 = 10.0**par[0]
    p1 = 10.0**par[1]
    alpha = par[2]
    D2 = ssp.distance.cdist(X1, X2, 'sqeuclidean')
    K = p0**2 * (1 + D2 / (2*alpha*p1**2))**(-alpha)
    return np.matrix(K)
def kernel_Per(X1,X2,par):
    p0 = 10.0**par[0]
    p1 = 10.0**par[1]
    period = par[2]
    DD = ssp.distance.cdist(X1, X2, 'euclidean')
    K = p0**2 * np.exp(- (np.sin(np.pi * DD / period))**2 / 2. / p1**2) 
    return np.matrix(K)
def kernel_QP(X1,X2,par):
    p0 = 10.0**par[0]
    p1 = 10.0**par[1]
    period = par[2]
    p3 = 10.0**par[3]
    DD = ssp.distance.cdist(X1, X2, 'euclidean')
    D2 = ssp.distance.cdist(X1, X2, 'sqeuclidean')
    K = p0**2 * np.exp(- (np.sin(np.pi * DD / period))**2 / 2. / p1**2 \
                          - D2 / 2. / p3**2)
    return np.matrix(K)
def add_wn(K,lsig):
    sigma=10.0**lsig
    N = K.shape[0]
    return K + sigma**2 * np.identity(N)
def get_kernel(name):
    if name == 'SE': return kernel_SE
    elif name == 'RQ': return kernel_RQ
    elif name == 'Mat32': return kernel_Mat32
    elif name == 'Per': return kernel_Per
    elif name == 'QP': return kernel_QP
    else: 
        print 'No kernel called %s - using SE' % name
        return kernel_SE

In [5]:
# Function to plot samples from kernel

def pltsamples(par0=0,par1=0,par2=1,wn=-3,kernel_shortname='SE'):
    x = np.r_[-5:5:101j]
    X = np.matrix([x]).T # scipy.spatial.distance expects matrices
    kernel = get_kernel(kernel_shortname)
    K = kernel(X,X,[par0,par1,par2])
    K = add_wn(K,wn)
    fig=pl.figure(figsize=(9,3))
    ax1 = pl.subplot2grid((1,3), (0, 0), aspect='equal')
    pl.imshow(np.sqrt(K),interpolation='nearest',vmin=0,vmax=10)
    pl.title('Covariance matrix')
    ax2 = pl.subplot2grid((1,3), (0,1),colspan=2)
    np.random.seed(0)
    for i in range(5):
        y = np.random.multivariate_normal(np.zeros(len(x)),K)
        pl.plot(x,y)
    pl.xlim(-5,5)
    pl.ylim(-10,10)
    pl.xlabel('x')
    pl.ylabel('y')
    pl.title('Samples from %s prior' % kernel_shortname)
    pl.tight_layout()
    return fig

In [6]:
StaticInteract(pltsamples, \
               par0=RangeWidget(-1,1,1,default=0),\
               par1=RangeWidget(-1,1,1,default=0))

## Some other commonly used kernels

* The Matern 3/2 kernel produces somewhat rougher behaviour

$$
k_{3/2}(x,x')= A^2 \left( 1 + \frac{\sqrt{3}r}{l} \right) \exp \left( - \frac{\sqrt{3}r}{l} \right).
$$

* The rational quadratic kernel is equivalent to a squared exponential with a powerlaw distribution of input scales

$$
k_{\rm RQ}(x,x') = A^2 \left(1 + \frac{r^2}{2 \alpha l} \right)^{-\alpha},
$$

* A periodic kernel can be constructed by replacing $r$ in any of the above by a periodic function of $r$. Here is an example based on the squared exponential:

$$
k_{\sin {\rm SE}}(x,x') = A^2 \exp \left[ -\frac{\sin(\pi r/P)}{2l^2} \right],
$$

* White noise can be included in any model by adding a constant to the diagonal elements of the covariance matrix.

$$
k_{\rm WN}(x,x') = \sigma^2 \delta(r)
$$

In [8]:
StaticInteract(pltsamples, \
               kernel_shortname=RadioWidget(['SE','Mat32','RQ','Per']))

In [9]:
StaticInteract(pltsamples, \
               kernel_shortname=RadioWidget(['RQ','Per']),\
               par2=RangeWidget(0.5,1.5,0.5,default=0))

## Custom covariance functions

To be a valid covariance matrix, an $N \times N$ matrix must be symmetric and positive semi-definite. 

If $k_1$ and $k_2$ are valid covariance functions, then $k_1+k_2$ and $k_1 \times k_2$ are also valid covariance functions. Thus new and almost arbitrarily complex covariance functions can be constructed from simple building blocks.

Example: quasi-periodic kernel with additive white noise.

$$
k(x,x') = A^2 \exp \left[ - \frac{\sin^2[\pi(x_1-x_2)/P]}{2L^2} - \frac{(x_1-x_2)^2}{2l^2} \right] + \sigma^2 \delta(x-x'),
$$

where $\delta(x)$ is the Kronecker delta function.

## Modelling data with GPs

GP regression enables us to model data by parametrising its covariance properties via the choice of kernel and its parameters.

Before we have any data, start with some knowledge of the kind of process we're going to observe, and hence the kind of functions we expect to see. 

This enables us to chose a covariance function $k$ and some initial values for its parameters $\boldsymbol{\phi}$. Together, these define a prior probability distribution over a family of model functions, which share the same covariance properties. 

Say we want to predict observations we plan to make at a set of test inputs $\boldsymbol{x}_*$. The prior distribution over the test outputs is simply

$$
p(\boldsymbol{y}_* \mid k,\phi) = \mathcal{N}( \boldsymbol{0},K_{**}),
$$

where $K_{**,ij} = k(x_{*,i},x_{*,j})$. 

Now imagine we have some training data $(\boldsymbol{x},\boldsymbol{y})$, drawn from the same process. The joint distribution over the training and test sets is

$$
p \left( \left[ \begin{array}{l} \boldsymbol{y} \\ \boldsymbol{y}_* \end{array} \right] \right) 
= \mathcal{N} \left( \left[ \begin{array}{l} 0 \\ 0 \end{array} \right], 
\left[ \begin{array}{ll} K & K_* \\ K_*^T & K_{**} \end{array} \right] \right),
$$

where $K_{ij} = k(x_i,x_j)$ and $K_{*,ij} = k(x_i,x_{*,j})$. 

The marginal distribution for the training set is known as the likelihood, as it represents the probability of observing the training set given the model:

$$ 
\mathcal{L} = p ( \boldsymbol{y} \mid k,\boldsymbol{\phi} ) = \mathcal{N} ( \boldsymbol{0},K).
$$

This is a measure of how good the model is at explaining the training set.

The conditional distribution for the test set given the training set is:

$$ 
p ( \boldsymbol{y}_* \mid \boldsymbol{y},k,\boldsymbol{\phi} ) = \mathcal{N} ( 
K_*^T K^{-1} \boldsymbol{y}, K_{**} - K_*^T K^{-1} K_* ).
$$

This is known as the predictive distribution, but it can be used for interpolating between observations as well as predicting future (or past!) observations. 

## Parameters and hyper-parameters

The parameters of the GP are the individual elements of the mean vector $\boldsymbol{m}$ and the covariance function $K$. Thus a GP has $N \times (N+1)$ parameters. 

That may seem like a lot. However, the power of Gaussian processes lies in the fact that marginalising over the individual parameters is analytic. We never have to deal with individual functions and instead perform inference over entire families of functions in one go.

The parameters of the covariance function (and of the mean function, if using one) are known as hyper-parameters of the GP. 

## Training the GP

In some applications, we care only about explaining the data as well as possible, and we might maximise the likelihood with respect to the hyper-parameters. 

# SWITCH TO PRESENTATION

## Full Bayesian treatment

In other applications, we may be interested in the marginal posterior distribtuion for one or more hyper-parameters, because they represent a physical quantity of interest. Or we may wish to evaluate the evidence for our model, e.g. to compare it to another model. Then we must evaluate the posterior distribution using e.g. MCMC techniques.

## The drawbacks

### Computational cost

GP regression involves inverting the covariance matrix and computing its determinant, which require $\mathcal{O}(N^3)$ operations. There are three main families of approaches get around this:
* Matrix inversion can be sped up under certain conditions (down to $\mathcal{O}(N \log N)$ in the best cases)
* Sub-sample the data
* "Sparse GPs" using Variational approximation

### Can be misleading

GP fits always looks good. It doesn't mean they are right! 

The GP is only ever as good as your choice of kernel & hyper-parameters

## Example 0: Mauna Kea CO$_2$ dataset

(From Rasmussen & Williams textbook)

<img src=images/RW_mauna_kea.png>

## Example 1: Exoplanet transmission spectra

Systematics due to telescope pointing and temperature variations dominate the planet's atmospheric signal.

Much controversy over the years...

Series of papers by Neale Gibson et al.: 
* [2011](http://adsabs.harvard.edu/abs/2011MNRAS.411.2199G): identifying the problem
* [2012a](http://adsabs.harvard.edu/abs/2012MNRAS.419.2683G): introducing the GP method
* [2012b](http://adsabs.harvard.edu/abs/2012MNRAS.422..753G): application to HST/WFC3
* 2013[a](http://adsabs.harvard.edu/abs/2013MNRAS.428.3680G), [b](http://adsabs.harvard.edu/abs/2013MNRAS.436.2974G): application to Gemini/GMOS
* [2014](http://adsabs.harvard.edu/abs/2014MNRAS.445.3401G): testing reliability of parametric and GP models

<img src=images/Gibson_transit_data.png>

<img src=images/Gibson_transit_pred.png>

<img src=images/Gibson_MCMC.png>

<img src=images/Gibson_transit_spectrum.png>

## Examples 2: The colour of an exoplanet 

* Tom Evans et al. (2013): *[The Deep Blue Color of HD 189733b: Albedo Measurements with Hubble Space Telescope/Space Telescope Imaging Spectrograph at Visible Wavelengths](http://adsabs.harvard.edu/abs/2013ApJ...772L..16E)*

<img src=images/Evans_HST.png>

<img src=images/Evans_HST_colour.png>

## Examples 3: Spitzer exoplanet transits and eclipses

* Tom Evans et al. (2015) *[A uniform analysis of HD 209458b Spitzer/IRAC light curves with Gaussian process models](http://adsabs.harvard.edu/abs/2015MNRAS.451.5199E)*

<img src=images/Evans_Spitzer_LC.jpg height=700>

## Example 4: Spotted star light curve

Star spots change the spectrum of a planet-host star, adversely affecting transmission spectrum and phase curve observations. To mitigate this, use a GP model the spot-induced flux variations in ground-based monitoring of the planet's host star.

* Aigrain, Pont & Zucker (2012) *[A simple method to estimate radial velocity variations due to stellar activity using photometry](http://adsabs.harvard.edu/abs/2012MNRAS.419.3147A)*
* Knutson et al. (2012) *[3.6 and 4.5 $\mu$m Phase Curves and Evidence for Non-equilibrium Chemistry in the Atmosphere of Extrasolar Planet HD 189733b](http://adsabs.harvard.edu/abs/2012ApJ...754...22K)*
* Sing et al. (2013) *[Hubble Space Telescope transmission spectroscopy of the exoplanet HD 189733b: high-altitude atmospheric haze in the optical and near-ultraviolet with STIS](http://adsabs.harvard.edu/abs/2011MNRAS.416.1443S)*
* Pont et al. (2013) *[The prevalence of dust on the exoplanet HD 189733b from Hubble and Spitzer observations](http://adsabs.harvard.edu/abs/2013MNRAS.432.2917P)*

<img src=images/HD189_full.png>

<img src=images/HD189_zoom.png>

<img src=images/HD189_MOST.png>

<img src=images/HD189_MOST_cond.png>

## Example 5: Disentangling planets and activity in radial velocity data

* Aigrain, Pont & Zucker (2012) *[A simple method to estimate radial velocity variations due to stellar activity using photometry](http://adsabs.harvard.edu/abs/2012MNRAS.419.3147A)*

* Rajpaul et al. (2015) *[A Gaussian process framework for modelling stellar activity signals in radial velocity data](http://adsabs.harvard.edu/abs/2015arXiv150607304R)*

<img src=images/rajpaul_framework.png>

<img src=images/rajpaul_SOAP.png>

<img src=images/rajpaul_aCen.png>

## Example 6: Modelling instrumental systematics in K2 data

* Aigrain et al. (2015) *[Precise time series photometry for the Kepler-2.0 mission](http://adsabs.harvard.edu/abs/2015MNRAS.447.2880A)*

<img src=images/K2.jpg>

## More info

### Textbooks

A good, detailed reference is [*Gaussian Processes for Machine Learning*](http://www.gaussianprocess.org/gpml/) by C. E. Rasmussen & C. Williams, MIT Press, 2006.

### Software

* [GPy](http://sheffieldml.github.io/GPy): general purpose python GP package. 
* [GPML](http://www.gaussianprocess.org/gpml/code/matlab/doc): general purpose Matlab GP package.
* [George](http://dan.iel.fm/george): GP regression using accelerated matrix inversion where possible.